In [1]:
import os
import csv
import numpy as np
import matplotlib.pyplot as plt

## Definición de la carga de datos

In [2]:
def cargar_datos_txt(fichero):
    path = os.path.join(os.getcwd(), fichero)
    lineas = csv.reader(open(path, newline='\n'))
    data = []
        
        
    for linea in lineas:
        fila = []
        for i in range(0, len(linea)):
            fila.append(int(linea[i]))
        data.append(fila)
        
    return data

In [3]:
def encuentra_posicion(valor, data):
    for i in range(0, len(data)):
        for j in range(0, len(data[i])): 
            
            if valor == data[i][j]:
                x, y = i,j
                break
    return x, y
            

In [4]:
def crear_matriz_recompensas(data, fin):
    
    cantidad = len(data)*len(data[0])
    
    lista = []
    for i in range(0, cantidad):
        lista.append([-1]*cantidad)
    
    #Definir vecinos
    for i in range(0, len(data)):
        
        for j in range(0, len(data[i])):
            actual = data[i][j]
            
        
            
            if len(data)  > i+1:
                vecino1 = data[i+1][j]
                if vecino1 == fin:
                    lista[actual][vecino1] = 100
                else:
                    lista[actual][vecino1] = 0
                    
            if 0 <= i-1:
                vecino2 = data[i-1][j]
                if vecino2 == fin:
                    lista[actual][vecino2] = 100
                else:
                    lista[actual][vecino2] = 0
                    
            if len(data[i]) > j+1:
                vecino3 = data[i][j+1]
                if vecino3 == fin:
                    lista[actual][vecino3] = 100
                else:
                    lista[actual][vecino3] = 0
                
            if 0 <= j-1:
                vecino4 = data[i][j-1]
                if vecino4 == fin:
                    lista[actual][vecino4] = 100
                else:
                    lista[actual][vecino4] = 0
            
    lista[fin][fin] = 100
    return lista
            
            
    

In [5]:
#Inicializa la matriz Q para el algorimo de Q-Learning, con los pesos de cada
#acción inicializados a 0
def inicializa_Q(recompensas):
   
    lista = []
    for i in range(0, len(recompensas)):
        lista.append([0]*len(recompensas))
        
     
    return lista
    

In [6]:
def carga_matriz_recompensas(fichero):
    datos = cargar_datos_txt(fichero)
    return datos

In [7]:
def cargar_datos():
    
    tipo = int(input('''Si quieres que se genere la matriz de recompensas a partir de un tablero pulsa 0,
si quieres cargar tu matriz de recompensas pulsa 1:
                        '''))
    
    inicio = int(input('Introduce el valor de la casilla de inicio: '))
    
    fin = int(input('Introduce el valor de la casilla objetivo: '))
    
    epochs = int(input('Introduce el número de episodios de entrenamiento: '))
    
    gamma = float(input('Introduce el valor del factor de aprendizaje gamma: '))
    
    epsilon = float(input('Introduce el valor del parámetro epsilon: '))
    
    alpha = float(input('Introduce el valor del parámetro alpha: '))
    
    if tipo == 0:
        
        data = cargar_datos_txt('./parametros.txt')
    
        recompensas = crear_matriz_recompensas(data, fin)
    
        q = inicializa_Q(recompensas)
        
    else:
        
        data = cargar_datos_txt('./parametros.txt')
        
        recompensas = cargar_datos_txt('./recompensas.txt')
        
        q = inicializa_Q(recompensas)
        
    
    return data, recompensas, q, inicio, fin, epochs, gamma, epsilon, alpha

## Definición de algoritmo de Q-Learning Fase 1

In [8]:
def seleccionar_accion_aleatoria(valor, recompensas):
    aleatorio = -1
    while aleatorio == -1:
        casilla = np.random.randint(len(recompensas),size=1)[0]
        aleatorio = recompensas[valor][casilla]
    return casilla    

In [9]:
def calcula_rendimiento(q):
    suma = 0
    maximo=1
    for i in range(0, len(q)):
        suma+=sum(q[i])
        if maximo < max(q[i]):
            maximo = max(q[i])
    return (suma/maximo)*100
    

In [10]:
def muestra_rendimiento(rendimiento):
    plt.title('Rendimiento del algoritmo de Q-Learning')
    plt.xlabel('Número de episodios de entrenamiento')
    plt.ylabel('Valor del rendimiento del algoritmo')
    plt.plot(rendimiento)
    plt.show()

In [11]:
def calcular_camino(q,inicio,fin):
    posicion = inicio
    camino = []
    camino.append(inicio)
    
    while posicion != fin:
        
        maximo = max(q[posicion])
        siguiente = q[posicion].index(maximo)
        camino.append(siguiente)
        posicion = siguiente
        
    representacion = ''
    
    for i in range(len(camino)):
        representacion += str(camino[i])+' -> '
        
    return representacion[:-4] 

In [12]:
def q_learning(data, recompensas, q, inicio, fin, epochs, gamma):
    rendimiento = []
    for epoch in range(0, epochs):
        
        #print('Epoca = '+str(epoch), 'Completado al '+str((epoch/epochs)*100)+'%')
        i = np.random.randint(len(data) ,size=1)[0]
        j = np.random.randint(len(data[i]),size=1)[0]
        estado = data[i][j]
        primero = True
        
        while (estado != fin) or (primero):
            
            accion = seleccionar_accion_aleatoria(estado, recompensas)
            maximo = max(q[accion])
            q[estado][accion] = recompensas[estado][accion] + gamma*maximo
            estado = accion
            primero = False
            
        rendimiento.append(calcula_rendimiento(q))
        
    camino = calcular_camino(q,inicio,fin)
    
    return q, rendimiento, camino
    

## Definición de algoritmo de Q-Learning Fase 2

In [13]:
def selecionar_accion_aleatoria_F2(estado, recompensas, q, epsilon):
    aleatorio = np.random.rand(1)[0]
    
    if aleatorio < epsilon:
        res = seleccionar_accion_aleatoria(estado, recompensas)
    else:
        maximo = max(q[estado])
        res = q[estado].index(maximo)
    return res

In [14]:
def q_learning_F2(data, recompensas, q, inicio, fin, epochs, gamma, epsilon, alpha):
    rendimiento = []
    for epoch in range(0, epochs):
        
        #print('Epoca = '+str(epoch), 'Completado al '+str((epoch/epochs)*100)+'%')
        i = np.random.randint(len(data) ,size=1)[0]
        j = np.random.randint(len(data[i]),size=1)[0]
        estado = data[i][j]
        primero = True
        
        while (estado != fin) or (primero):
            
            accion = selecionar_accion_aleatoria_F2(estado, recompensas, q , epsilon)
            maximo = max(q[accion])
            q[estado][accion] = recompensas[estado][accion] + gamma*maximo
            estado = accion
            primero = False
            
        rendimiento.append(calcula_rendimiento(q))
        epsilon *= alpha
        
    camino = calcular_camino(q, inicio, fin)
    
    return q, rendimiento, camino

# Main:

In [21]:
def main(): 
    data, recompensas, q, inicio, fin, epochs, gamma, epsilon, alpha = cargar_datos()
    q, rendimiento, camino  = q_learning(data, recompensas, q, inicio, fin, epochs, gamma)
    print('\nMatriz Q obtenida =\n',q)
    muestra_rendimiento(rendimiento)
    
    print('El camino óptimo es: ' + camino)
    
if __name__ == "__main__": main()
   

Si quieres que se genere la matriz de recompensas a partir de un tablero pulsa 0,
si quieres cargar tu matriz de recompensas pulsa 1:
                        0
Introduce el valor de la casilla de inicio: 0
Introduce el valor de la casilla objetivo: 6
Introduce el número de episodios de entrenamiento: 10
Introduce el valor del factor de aprendizaje gamma: 1
Introduce el valor del parámetro epsilon: 1
Introduce el valor del parámetro alpha: 1


KeyboardInterrupt: 

In [ ]:
data, recompensas, q, inicio, fin, epochs, gamma, epsilon, alpha = cargar_datos()
q, rendimiento,  = q_learning(data, recompensas, q, inicio, fin, epochs, gamma)

Si quieres que se genere la matriz de recompensas a partir de un tablero pulsa 0,
si quieres cargar tu matriz de recompensas pulsa 1:
                        0
Introduce el valor de la casilla de inicio: 1
Introduce el valor de la casilla objetivo: 12
Introduce el número de episodios de entrenamiento: 12
Introduce el valor del factor de aprendizaje gamma: 1
Introduce el valor del parámetro epsilon: 1
Introduce el valor del parámetro alpha: 1


In [ ]:
for i in range(len(recompensas)):
    print(str(i)+':',recompensas[i])

In [ ]:
'''
i = 0.1
while i<=1:
    for b in range(0,4):
        q = inicializa_Q(data)
        q, rendimiento = q_learning_F2(data, recompensas, q, 0, 6, 50, 0.5, 1, i)
        print('i = ', i)
        muestra_rendimiento(rendimiento)
    i+=0.1
'''